# 说明：

前面的“双向RNN”处理“自然语言”问题时有可能比原始RNN效果更好；这里的“一维卷积”处理“时间序列”问题时可以媲美RNN，但是计算量要小非常非常多！！

一维卷积进行“特征提取”时，它对特征的提取要比传统的“信号处理”要好很多！因为它提取的特征是很全的、甚至是更加高维、抽象的特征。另外，卷积操作可以识别“**局部模式**” —— 具有“**平移不变性**” —— 即：一个特征，比如一只猫（只要特征没有被改变），不论位置在哪，都能把它学/识别出来！

---
一维卷积也有“池化层”：

补充：如何理解“池化层”作用(一维、二维都适用这种理解) —— 池化层不断把原始数据(图像/序列)变小，也就是不断把无效内容剔除不断保留有效特征的过程。在数据变小(池化)的过程中，卷积核大小不变，那其实就间接的增大了卷积层对图像整体的“**感受视野**”（卷积核相对现在的图像，它变大了）！让**越靠后面**的卷积层越看到“**全局、抽象的特征**”（虽然图小了，但图小是因为把无效内容剔除了，有效特征仍保留，所以没影响）。

所以：在一维卷积网络中，卷积层对序列数据进行平移不变性的特征提取（比如能反应出情绪的词语），池化层不断把无效内容剔除来减小序列长度（比如不能反应情绪的一些主语、感叹词、连词等）。虽然卷积核大小不变（一直比如只能看5个词），随着序列把池化层减小，卷积核反而相对地越来越能看到“**更广、更大、更全局、更抽象的关键特征(比如：“不”(否定) + “喜欢”(肯定) = “不喜欢”(否) )”！** 

一维卷积网络，通过“**卷积对特征提取平移不变性 + 池化层剔除无效数据将序列浓缩(间接增大卷积层的感受野)”**，换了个角度弥补了自己不能像RNN那样综合前面出现过的数据。

---
输入数据的区别：
- 二维卷积conv2d，其输入数据是4维的：(batch, height, width, channels)
- 一维卷积conv1d，其数据数据是3维的：(batch, time_step, features) —— 和LSTM一样。

卷积核的区别：
- 二维卷积核：一般就是3x3，比较小
- 一维卷积核：可以大一些（比如7,9,11,13等）；因为如果每次只看3个词，确实可能因为太局限而导致判断出现问题！

# 数据处理：和前面38一样

In [1]:
import keras
from keras import layers
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
data = pd.read_csv('E:/Python_code/keras_total/日月光华-keras课程资料/航空公司Tweets评论数据/Tweets.csv')

In [3]:
# 数据提取：
data = data[ ['airline_sentiment','text'] ]

In [4]:
# 提取数据：
data_p = data[data.airline_sentiment == 'positive']
data_n = data[data.airline_sentiment == 'negative']
data_n = data_n.iloc[:len(data_p)]  # negative数据量和“positive”的一样多！

In [5]:
# 合并 + 打散：
data = pd.concat([data_n, data_p])
data = data.sample( len(data) )

In [6]:
# 数值化：
data['review'] = (data.airline_sentiment == 'positive').astype('int')  # 是positive的转为1，是negative转为0
del data['airline_sentiment']

In [7]:
data.head()

,text,review
3018,@united my flight Cancelled Flightled new flig...,0
1117,@united How do I formally complain about your ...,0
2704,"@united not tonight, we got a hotel and after ...",0
1931,@united thank you for fully boarding flight 16...,0
4903,"@SouthwestAir Yes, please.",1


In [8]:
# 文本词嵌入：
import re

token = re.compile('[A-Za-z]+|[!?,.()]')

# 定义的函数1：把上面定义的正则化规则，适用于所输入的数据；并把大小字母转为小写：
def reg_text(text):
    new_text = token.findall(text)  # 制定的规则处理输入的文本
    new_text = [word.lower() for word in new_text]  # 把大写字母转为小写
    return new_text  # 返回处理后的新文本

data['text'] = data.text.apply(reg_text)  # 应用上面的函数： 

In [9]:
data.head()

,text,review
3018,"[united, my, flight, cancelled, flightled, new...",0
1117,"[united, how, do, i, formally, complain, about...",0
2704,"[united, not, tonight, ,, we, got, a, hotel, a...",0
1931,"[united, thank, you, for, fully, boarding, fli...",0
4903,"[southwestair, yes, ,, please, .]",1


In [10]:
# set数据类型中，虽然没有索引，但是内容不重复！很利于进行总数的统计。
word_set = set()

# 循环统计：
for text in data.text:
    for word in text:
        word_set.add(word) # 如果前后有重复的东西出现，这里不会录入
        
maxword = len(word_set) + 1  # 7100 + 1 

In [11]:
# 获得每个单词的向量：
word_list = list(word_set)  
word_list.index('spending'), word_list.index(',')

(4654, 4208)

In [12]:
word_index = dict( (word, word_list.index(word) + 1) for word in word_list )

# 把评论转换为单词的编号列表/向量：
data_ok = data.text.apply(lambda x: [word_index.get(word, 0) for word in x])

maxlen = max( len(x) for x in data_ok ) # 40

In [13]:
# 0值填充：
data_ok = keras.preprocessing.sequence.pad_sequences( data_ok.values, maxlen = maxlen )

In [14]:
# 训练数据与对应的标签：
x_train = data_ok
x_lable = data.review.values

In [15]:
data_ok.shape

(4726, 40)

# 网络搭建：一维卷积神经网络

In [16]:
model = keras.Sequential()

In [17]:
model.add( layers.Embedding(maxword, 50, input_length = maxlen) )

In [18]:
# 一般只用管下面3个参数：padding随意
model.add( layers.Conv1D( filters = 32, kernel_size = 7, activation='relu', padding = 'same') )
model.add( layers.MaxPooling1D( pool_size = 3) ) # 变为原来的1/3
model.add( layers.Conv1D( filters = 32, kernel_size = 7, activation='relu', padding = 'same') )
model.add( layers.GlobalAveragePooling1D() )     # 全局池化，相当于layers.Flatten()，把数据展平

In [19]:
model.summary()
# (13, 32) ——> (32) 把13个32数据，进行一个全局平均(对应位置的13个数据求平均)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 50)            355050    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40, 32)            11232     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 32)            7200      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
Total params: 373,482
Trainable params: 373,482
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.add( layers.Dense(1, activation='sigmoid') )

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 50)            355050    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40, 32)            11232     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 32)            7200      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 373,515
Trainable params: 373,515
Non-trainable params: 0
________________________________________________

In [22]:
model.compile( optimizer = keras.optimizers.Adam(),
               loss = 'binary_crossentropy',
               metrics = ['acc']
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
history = model.fit( x_train, x_lable, epochs=10, batch_size=128, validation_split=0.2 )
# 运行速度巨快！且在测试集上的精度基本没差距！

Train on 3780 samples, validate on 946 samples
Epoch 1/10
3780/3780 [==============================] - 1s 149us/step - loss: 0.0092 - acc: 0.9984 - val_loss: 0.2794 - val_acc: 0.9239
Epoch 2/10
3780/3780 [==============================] - 1s 159us/step - loss: 0.0061 - acc: 0.9992 - val_loss: 0.2873 - val_acc: 0.9281
Epoch 3/10
3780/3780 [==============================] - 1s 156us/step - loss: 0.0052 - acc: 0.9997 - val_loss: 0.2997 - val_acc: 0.9197
Epoch 4/10
3780/3780 [==============================] - 1s 155us/step - loss: 0.0044 - acc: 1.0000 - val_loss: 0.3064 - val_acc: 0.9271
Epoch 5/10
3780/3780 [==============================] - 1s 161us/step - loss: 0.0035 - acc: 1.0000 - val_loss: 0.3156 - val_acc: 0.9271
Epoch 6/10
3780/3780 [==============================] - 1s 174us/step - loss: 0.0031 - acc: 0.9997 - val_loss: 0.3237 - val_acc: 0.9249
Epoch 7/10
3780/3780 [==============================] - 1s 169us/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.3338 - val_acc: 0.9239
E

说明：精度上基本无差，但是速度比LSTM要快居多(因为LSTM中循环操作很费时间)！所以一维卷积用于“翻译”很好。